In [2]:
from selenium import webdriver
import time
import requests
import pandas as pd
import openpyxl
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm_notebook

from datetime import date
from datetime import timedelta


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *

In [46]:
start_time = time.time()

# 날짜 필터링 전 모든 데이터를 담는 곳

soap2day_id_list_all = []
title_list_all = []
year_list_all = []
time_list_all = []
overview_list_all = []
director_list_all = []
starring_list_all = []
url_list_all = []
genre_list_all = []
studio_list_all = []

# 날짜 필터링 된 후 최신 데이터를 담는 곳

soap2day_id_list = []
title_list = []
year_list = []
time_list = []
overview_list = []
director_list = []
starring_list = []
url_list = []
genre_list = []
studio_list = []

# 날짜 필터링 된 후 개봉예정 데이터를 담는 곳

soap2day_id_list_2 = []
title_list_2 = []
year_list_2 = []
time_list_2 = []
overview_list_2 = []
director_list_2 = []
starring_list_2 = []
url_list_2 = []
genre_list_2 = []
studio_list_2 = []


# 날짜 필터링을 위한 리스트

release_date_all = []

# 시작

path_ = '../../chromedriver/chromedriver'

url = 'https://soap2day.to/movielist/sort/release/year/2020?page=1'

driver = webdriver.Chrome(path_)

driver.get(url)

time.sleep(7)

soup = BeautifulSoup(driver.page_source)

page_counter = []
page_cnt = 1
title_cnt = 1

# 최대 페이지 넘버를 가져오기

while True:
    try:
        page_num_finder = driver.find_element_by_xpath('/html/body/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[3]/ul/li[' + str(page_cnt + 2) + ']/a')
    except NoSuchElementException:
        break
    page_counter.append(page_num_finder.text)
    page_cnt += 1
    
max_page = int(page_counter[-2])
print('Last Page Number : ', max_page)

# page 1부터 최대 페이지까지 모든 href를 가져오고, 그것을 주소 리스트에 삽입

while True:
    try:
        title = driver.find_element_by_xpath('/html/body/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[1]/div/div[' + str(title_cnt) + ']/div/div[2]/h5/a')
        title_list_all.append(title.text.strip())
    except NoSuchElementException:
        break
    url_list_all.append(title.get_attribute('href'))
    title_cnt += 1

for i in range(2, max_page + 1):
    title_cnt = 1
    url = 'https://soap2day.to/movielist/sort/release/year/2020?page=' + str(i)
    driver.get(url)
    time.sleep(3)
    
    while True:
        try:
            title = driver.find_element_by_xpath('/html/body/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[1]/div/div[' + str(title_cnt) + ']/div/div[2]/h5/a')
            title_list_all.append(title.text.strip())
        except NoSuchElementException:
            break
        url_list_all.append(title.get_attribute('href'))
        title_cnt += 1

crawling_counter = 0
        
for i in url_list_all:
    driver.get(i)
    time.sleep(3)
    crawling_counter += 1
    
    
    # 영화 고유 id값
    
    id_loc_1 = i.find('movie')
    id_loc_2 = i.find('html')
    
    soap2day_id_list_all.append(i[id_loc_1 + 6:id_loc_2 - 1])
    
    # 감독
    
    temp_director = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[3]/div[1]/div/div/div/div[1]/p[3]')
    director_list_all.append(temp_director.text.strip())
    
    # 출연 배우
    
    temp_starring = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[3]/div[1]/div/div/div/div[1]/p[6]')
    starring_list_all.append(temp_starring.text.strip())
    
    # 장르
    
    temp_genre = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[3]/div[1]/div/div/div/div[1]/p[9]')
    genre_list_all.append(temp_genre.text.strip())
    
    # 출시년도, indexing을 위한 출시일 년,월,일로 슬라이싱
    
    temp_release = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[3]/div[1]/div/div/div/div[1]/p[12]')
    release_tmp = temp_release.text.strip()
    try:
        release_tmp_year = int(release_tmp[:4])
        release_tmp_month = int(release_tmp[5:7])
        release_tmp_day = int(release_tmp[8:])
    except ValueError:
        year_list_all.append('None')
        release_date_all.append('None')
    else:
        year_list_all.append(release_tmp_year)
        release_date_all.append(date(release_tmp_year, release_tmp_month, release_tmp_day))
    
    # 개요
    
    temp_overview = driver.find_element_by_xpath('//*[@id="wrap"]')
    overview_list_all.append(temp_overview.text.strip())
    
    if crawling_counter % 10 == 0:
        print(str(crawling_counter), ' movies done in progress...')
    
print('All Crawling Ended...')

print('Start Indexing New And Upcoming Data...')

recent_index = []
upcoming_index = []

# indexing

for i in range(len(release_date_all)):
    if release_date_all[i] != 'None':
        if (release_date_all[i] > date.today() - timedelta(days = 60)) and (release_date_all[i] < date.today() + timedelta(days = 7)):
            recent_index.append(i)
        elif (release_date_all[i] >= date.today() + timedelta(days = 7)):
            upcoming_index.append(i)
        
# 최신 영화 기준에 맞는 것들 삽입        
        
for i in recent_index:
    soap2day_id_list.append(soap2day_id_list_all[i])
    title_list.append(title_list_all[i])
    year_list.append(year_list_all[i])
    overview_list.append(overview_list_all[i])
    director_list.append(director_list_all[i])
    starring_list.append(starring_list_all[i])
    url_list.append(url_list_all[i])
    genre_list.append(genre_list_all[i])
    
# 개봉 예정 영화 기준에 맞는 것들 삽입

for i in upcoming_index:
    soap2day_id_list_2.append(soap2day_id_list_all[i])
    title_list_2.append(title_list_all[i])
    year_list_2.append(year_list_all[i])
    overview_list_2.append(overview_list_all[i])
    director_list_2.append(director_list_all[i])
    starring_list_2.append(starring_list_all[i])
    url_list_2.append(url_list_all[i])
    genre_list_2.append(genre_list_all[i])
    
print('All Crawling Ended...')


items_recent = pd.DataFrame({'soap2day_id' : soap2day_id_list, 'title' : title_list, 'release' : year_list, 
                      'genres' : genre_list, 'director' : director_list, 'actor' : starring_list,
                      'country' : np.nan, 'runtime' : np.nan, 'production' : np.nan, 'overview' : overview_list,
                      'url' : url_list})

items_upcoming = pd.DataFrame({'soap2day_id' : soap2day_id_list_2, 'title' : title_list_2, 'release' : year_list_2, 
                      'genres' : genre_list_2, 'director' : director_list_2, 'actor' : starring_list_2,
                      'country' : np.nan, 'runtime' : np.nan, 'production' : np.nan, 'overview' : overview_list_2,
                      'url' : url_list_2})

items_recent.to_excel('SOAP2DAY_Crawling_Recent.xlsx')
items_upcoming.to_excel('SOAP2DAY_Crawling_Upcoming.xlsx')


print('Spend Time : ', time.time() - start_time)

driver.close()

Last Page Number :  5
10  movies done in progress...
20  movies done in progress...
30  movies done in progress...
40  movies done in progress...
50  movies done in progress...
60  movies done in progress...
70  movies done in progress...
80  movies done in progress...
90  movies done in progress...
100  movies done in progress...
110  movies done in progress...
120  movies done in progress...
All Crawling Ended...
Start Indexing New And Upcoming Data...
All Crawling Ended...
Spend Time :  1289.1455788612366
